# Advanced Espresso AI Training Notebook

This notebook provides advanced training capabilities for the Espresso AI system using TensorFlow, scikit-learn, and other ML libraries for sophisticated model development and analysis.

## Features:
- **Real Machine Learning Models**: Neural networks, regression, classification
- **Advanced Feature Engineering**: Comprehensive feature extraction and selection
- **Model Evaluation**: Cross-validation, performance metrics, visualization
- **Data Analysis**: Statistical analysis, pattern recognition, correlation studies
- **Export Models**: Save trained models for production use


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

print("📚 Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


In [ ]:
# Data Loading and Preprocessing
# This cell simulates loading coffee log data from your database
# In practice, you would connect to MongoDB and fetch real data

def create_sample_data(n_samples=1000):
    """Create realistic sample coffee log data for training"""
    np.random.seed(42)
    
    data = {
        'grindSize': np.random.normal(15, 3, n_samples).clip(5, 25),
        'extractionTime': np.random.normal(30, 5, n_samples).clip(15, 50),
        'temperature': np.random.normal(93, 2, n_samples).clip(85, 96),
        'inWeight': np.random.normal(18, 2, n_samples).clip(12, 25),
        'outWeight': np.random.normal(36, 4, n_samples).clip(20, 50),
        'usedPuckScreen': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),
        'usedWDT': np.random.choice([0, 1], n_samples, p=[0.4, 0.6]),
        'usedPreInfusion': np.random.choice([0, 1], n_samples, p=[0.5, 0.5]),
        'preInfusionTime': np.random.normal(5, 2, n_samples).clip(0, 15),
        'roastLevel': np.random.choice(['light', 'medium', 'dark'], n_samples),
        'processMethod': np.random.choice(['washed', 'natural', 'honey'], n_samples)
    }
    
    # Calculate derived features
    data['ratio'] = data['outWeight'] / data['inWeight']
    data['flowRate'] = data['outWeight'] / data['extractionTime']
    
    # Generate realistic shot quality based on parameters
    quality = np.zeros(n_samples)
    for i in range(n_samples):
        base_quality = 5
        
        # Extraction time bonus
        if 25 <= data['extractionTime'][i] <= 35:
            base_quality += 1
        elif data['extractionTime'][i] < 20 or data['extractionTime'][i] > 40:
            base_quality -= 1
            
        # Ratio bonus
        if 1.8 <= data['ratio'][i] <= 2.2:
            base_quality += 1
        elif data['ratio'][i] < 1.5 or data['ratio'][i] > 2.5:
            base_quality -= 1
            
        # Temperature bonus
        if 90 <= data['temperature'][i] <= 95:
            base_quality += 0.5
            
        # Technique bonuses
        if data['usedPuckScreen'][i]:
            base_quality += 0.5
        if data['usedWDT'][i]:
            base_quality += 0.5
        if data['usedPreInfusion'][i]:
            base_quality += 0.5
            
        # Add some randomness
        base_quality += np.random.normal(0, 0.5)
        
        quality[i] = np.clip(base_quality, 1, 10)
    
    data['shotQuality'] = quality
    
    return pd.DataFrame(data)

# Create sample dataset
df = create_sample_data(1000)
print(f"📊 Created dataset with {len(df)} samples")
print(f"📈 Quality distribution:")
print(df['shotQuality'].value_counts().sort_index())
print(f"\n📊 Dataset shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")


In [ ]:
# Data Visualization and Analysis
plt.figure(figsize=(15, 10))

# Quality distribution
plt.subplot(2, 3, 1)
df['shotQuality'].hist(bins=10, alpha=0.7, color='brown')
plt.title('Shot Quality Distribution')
plt.xlabel('Quality Score')
plt.ylabel('Frequency')

# Extraction time vs Quality
plt.subplot(2, 3, 2)
plt.scatter(df['extractionTime'], df['shotQuality'], alpha=0.6, color='green')
plt.title('Extraction Time vs Quality')
plt.xlabel('Extraction Time (s)')
plt.ylabel('Quality Score')

# Ratio vs Quality
plt.subplot(2, 3, 3)
plt.scatter(df['ratio'], df['shotQuality'], alpha=0.6, color='orange')
plt.title('Ratio vs Quality')
plt.xlabel('Ratio (out/in)')
plt.ylabel('Quality Score')

# Temperature vs Quality
plt.subplot(2, 3, 4)
plt.scatter(df['temperature'], df['shotQuality'], alpha=0.6, color='red')
plt.title('Temperature vs Quality')
plt.xlabel('Temperature (°C)')
plt.ylabel('Quality Score')

# Grind size vs Quality
plt.subplot(2, 3, 5)
plt.scatter(df['grindSize'], df['shotQuality'], alpha=0.6, color='purple')
plt.title('Grind Size vs Quality')
plt.xlabel('Grind Size')
plt.ylabel('Quality Score')

# Correlation heatmap
plt.subplot(2, 3, 6)
correlation_matrix = df[['grindSize', 'extractionTime', 'temperature', 'inWeight', 'outWeight', 'ratio', 'shotQuality']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')

plt.tight_layout()
plt.show()

# Statistical summary
print("📊 Statistical Summary:")
print(df.describe())


In [ ]:
# Feature Engineering and Preprocessing
def prepare_features(df):
    """Prepare features for machine learning"""
    # Encode categorical variables
    le_roast = LabelEncoder()
    le_process = LabelEncoder()
    
    df_encoded = df.copy()
    df_encoded['roastLevel_encoded'] = le_roast.fit_transform(df['roastLevel'])
    df_encoded['processMethod_encoded'] = le_process.fit_transform(df['processMethod'])
    
    # Select features for training
    feature_columns = [
        'grindSize', 'extractionTime', 'temperature', 'inWeight', 'outWeight',
        'usedPuckScreen', 'usedWDT', 'usedPreInfusion', 'preInfusionTime',
        'roastLevel_encoded', 'processMethod_encoded', 'ratio', 'flowRate'
    ]
    
    X = df_encoded[feature_columns]
    y = df_encoded['shotQuality']
    
    return X, y, feature_columns

# Prepare features
X, y, feature_columns = prepare_features(df)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"📊 Training set: {X_train.shape[0]} samples")
print(f"📊 Test set: {X_test.shape[0]} samples")
print(f"📊 Features: {len(feature_columns)}")
print(f"📋 Feature names: {feature_columns}")


In [ ]:
# Traditional Machine Learning Models
def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    """Train and evaluate multiple ML models"""
    
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Lasso Regression': Lasso(alpha=0.1),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'Neural Network': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"🔄 Training {name}...")
        
        # Train model
        model.fit(X_train_scaled, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test_scaled)
        
        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        # Cross-validation score
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
        cv_mae = -cv_scores.mean()
        
        results[name] = {
            'model': model,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'cv_mae': cv_mae,
            'predictions': y_pred
        }
        
        print(f"✅ {name} - MAE: {mae:.3f}, RMSE: {rmse:.3f}, R²: {r2:.3f}, CV MAE: {cv_mae:.3f}")
    
    return results

# Train traditional ML models
ml_results = train_and_evaluate_models(X_train, X_test, y_train, y_test)


In [ ]:
# Deep Learning with TensorFlow/Keras
def create_deep_learning_model(input_shape):
    """Create a deep neural network for coffee quality prediction"""
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_shape,)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='linear')  # Regression output
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    return model

# Create and train deep learning model
print("🧠 Creating Deep Learning Model...")
dl_model = create_deep_learning_model(X_train_scaled.shape[1])
dl_model.summary()

# Train the model
print("🔄 Training Deep Learning Model...")
history = dl_model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    verbose=1
)

# Evaluate deep learning model
dl_predictions = dl_model.predict(X_test_scaled).flatten()
dl_mae = mean_absolute_error(y_test, dl_predictions)
dl_rmse = np.sqrt(mean_squared_error(y_test, dl_predictions))
dl_r2 = r2_score(y_test, dl_predictions)

print(f"✅ Deep Learning Model - MAE: {dl_mae:.3f}, RMSE: {dl_rmse:.3f}, R²: {dl_r2:.3f}")


In [ ]:
# Model Comparison and Visualization
def plot_training_history(history):
    """Plot training history for deep learning model"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss
    ax1.plot(history.history['loss'], label='Training Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    
    # MAE
    ax2.plot(history.history['mae'], label='Training MAE')
    ax2.plot(history.history['val_mae'], label='Validation MAE')
    ax2.set_title('Model MAE')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('MAE')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

def compare_model_performance(ml_results, dl_mae, dl_rmse, dl_r2):
    """Compare performance of all models"""
    # Add deep learning results
    ml_results['Deep Learning'] = {
        'mae': dl_mae,
        'rmse': dl_rmse,
        'r2': dl_r2
    }
    
    # Create comparison DataFrame
    comparison_data = []
    for name, results in ml_results.items():
        comparison_data.append({
            'Model': name,
            'MAE': results['mae'],
            'RMSE': results['rmse'],
            'R²': results['r2']
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.sort_values('MAE')
    
    print("📊 Model Performance Comparison:")
    print(comparison_df.to_string(index=False))
    
    # Plot comparison
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # MAE comparison
    ax1.bar(comparison_df['Model'], comparison_df['MAE'], color='skyblue')
    ax1.set_title('Mean Absolute Error Comparison')
    ax1.set_ylabel('MAE')
    ax1.tick_params(axis='x', rotation=45)
    
    # RMSE comparison
    ax2.bar(comparison_df['Model'], comparison_df['RMSE'], color='lightcoral')
    ax2.set_title('Root Mean Square Error Comparison')
    ax2.set_ylabel('RMSE')
    ax2.tick_params(axis='x', rotation=45)
    
    # R² comparison
    ax3.bar(comparison_df['Model'], comparison_df['R²'], color='lightgreen')
    ax3.set_title('R² Score Comparison')
    ax3.set_ylabel('R²')
    ax3.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return comparison_df

# Plot training history
plot_training_history(history)

# Compare all models
comparison_df = compare_model_performance(ml_results, dl_mae, dl_rmse, dl_r2)


In [ ]:
# Model Export and Production Integration
def export_best_model(comparison_df, ml_results, dl_model, scaler, feature_columns):
    """Export the best performing model for production use"""
    
    # Get best model based on MAE
    best_model_name = comparison_df.iloc[0]['Model']
    print(f"🏆 Best Model: {best_model_name}")
    
    if best_model_name == 'Deep Learning':
        # Save TensorFlow model
        dl_model.save('best_coffee_quality_model.h5')
        print("✅ Deep Learning model saved as 'best_coffee_quality_model.h5'")
        
        # Save scaler
        import joblib
        joblib.dump(scaler, 'coffee_scaler.pkl')
        print("✅ Scaler saved as 'coffee_scaler.pkl'")
        
        # Save feature columns
        import json
        with open('feature_columns.json', 'w') as f:
            json.dump(feature_columns, f)
        print("✅ Feature columns saved as 'feature_columns.json'")
        
    else:
        # Save sklearn model
        best_model = ml_results[best_model_name]['model']
        import joblib
        joblib.dump(best_model, 'best_coffee_quality_model.pkl')
        joblib.dump(scaler, 'coffee_scaler.pkl')
        
        with open('feature_columns.json', 'w') as f:
            json.dump(feature_columns, f)
        
        print(f"✅ {best_model_name} model saved as 'best_coffee_quality_model.pkl'")
        print("✅ Scaler saved as 'coffee_scaler.pkl'")
        print("✅ Feature columns saved as 'feature_columns.json'")

def create_production_predictor():
    """Create a production-ready predictor function"""
    
    predictor_code = '''
# Production Coffee Quality Predictor
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
import json

class CoffeeQualityPredictor:
    def __init__(self, model_path, scaler_path, features_path):
        self.model = joblib.load(model_path)
        self.scaler = joblib.load(scaler_path)
        with open(features_path, 'r') as f:
            self.feature_columns = json.load(f)
    
    def predict_quality(self, shot_data):
        """Predict coffee shot quality"""
        # Prepare feature vector
        features = np.array([[
            shot_data.get('grindSize', 15),
            shot_data.get('extractionTime', 30),
            shot_data.get('temperature', 93),
            shot_data.get('inWeight', 18),
            shot_data.get('outWeight', 36),
            shot_data.get('usedPuckScreen', 0),
            shot_data.get('usedWDT', 0),
            shot_data.get('usedPreInfusion', 0),
            shot_data.get('preInfusionTime', 0),
            shot_data.get('roastLevel_encoded', 1),
            shot_data.get('processMethod_encoded', 0),
            shot_data.get('ratio', 2.0),
            shot_data.get('flowRate', 1.2)
        ]])
        
        # Scale features
        features_scaled = self.scaler.transform(features)
        
        # Make prediction
        quality = self.model.predict(features_scaled)[0]
        return max(1, min(10, round(quality)))

# Usage example:
# predictor = CoffeeQualityPredictor('best_coffee_quality_model.pkl', 'coffee_scaler.pkl', 'feature_columns.json')
# quality = predictor.predict_quality({
#     'grindSize': 15,
#     'extractionTime': 30,
#     'temperature': 93,
#     'inWeight': 18,
#     'outWeight': 36,
#     'usedPuckScreen': 1,
#     'usedWDT': 1,
#     'usedPreInfusion': 0,
#     'preInfusionTime': 0,
#     'roastLevel_encoded': 1,
#     'processMethod_encoded': 0,
#     'ratio': 2.0,
#     'flowRate': 1.2
# })
'''
    
    with open('coffee_quality_predictor.py', 'w') as f:
        f.write(predictor_code)
    
    print("✅ Production predictor code saved as 'coffee_quality_predictor.py'")

# Export best model
export_best_model(comparison_df, ml_results, dl_model, scaler, feature_columns)

# Create production predictor
create_production_predictor()

print("\n🎯 Training Complete! Your AI model is ready for production.")
print("📁 Files created:")
print("   - best_coffee_quality_model.h5/pkl (trained model)")
print("   - coffee_scaler.pkl (feature scaler)")
print("   - feature_columns.json (feature names)")
print("   - coffee_quality_predictor.py (production code)")
